In [1]:
import importlib
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfiltfilt

import LocomotionAnalysisTools as lat

In [2]:
# Reload import if modified (jupyter notebook caches imports by default)
importlib.reload(lat)

<module 'LocomotionAnalysisTools' from 'C:\\Users\\robaeten\\OneDrive - UGent\\Documenten\\Werk\\python\\GMC_GPR_GDA\\LocomotionAnalysisTools.py'>

In [3]:
# Define where to get the data
dataFolder = "C:/Users/robaeten/OneDrive - UGent/Gait Classification and References/Data"

# subjectSessions = {"Subject1": ["Session4"]}
# subjectSessions = {"Subject2": ["Session1"]}
# subjectSessions = {"Subject3": ["Session1"]}
# subjectSessions = {"Subject4": ["Session1"]}
# subjectSessions = {"Subject5": ["Session1"]}
# subjectSessions = {"Subject6": ["Session1"]}
# subjectSessions = {"Subject7": ["Session1"]}
# subjectSessions = {"Subject8": ["Session1"]}
# subjectSessions = {"Subject9": ["Session1"]}
# subjectSessions = {"Subject10": ["Session1"]}
# subjectSessions = {"Subject11": ["Session1"]}
# subjectSessions = {"Subject12": ["Session1"]}
# subjectSessions = {"Subject13": ["Session1"]}
# subjectSessions = {"Subject14": ["Session1"]}
# subjectSessions = {"Subject15": ["Session1"]}
# subjectSessions = {"Subject16": ["Session1"]}
# subjectSessions = {"Subject17": ["Session1"]}
# subjectSessions = {"Subject18": ["Session1"]}
# subjectSessions = {"Subject19": ["Session1"]}
subjectSessions = {"Subject20": ["Session1"]}

dataToRead = "_Proc_Ref"
foldersToRead, _ = lat.get_data_locations(dataFolder, subjectSessions, dataToRead)

In [4]:
def get_filenames_mod_out(foldersToRead):
    filesToRead = []
    
    # Loop over all the measurement sessions
    for i, folder in enumerate(foldersToRead):
        fileList = os.listdir(folder)
        filesChecked = []
        
        # Loop over all the different activities and their iterations
        for file in fileList:
            splitFileName = file.split(" - ")
            activity = splitFileName[2]
            
            # To not repeat same activity
            try:
                if (activity[-3] == "(") and (activity[-1] == ")") and (int(activity[-2]) in range(10)):
                    activity = activity[:-3]
            except:
                pass
                
            splitFileName[-1] = splitFileName[-1].split(".")[0] # Remove extension
            fileNameBase = " - ".join(splitFileName[0:4])
    
            # Update list of checked sessions
            if (fileNameBase not in filesChecked):
                filesChecked.append(fileNameBase)
        
                # Find the right flags
                flags = ""
                if " - badMarker" in file:
                    flags = flags + " - badMarker"
                if " - badForcePlate" in file:
                    flags = flags + " - badForcePlate"
                if " - badStep" in file:
                    flags = flags + " - badStep"
                
                # Get file and update
                fileModelOutputs = folder + "/" + fileNameBase + " - ModelOutputs" + flags + ".csv"
                filesToRead.append(fileModelOutputs)

    return filesToRead

In [5]:
filesToRead = get_filenames_mod_out(foldersToRead)
print(filesToRead[0])

C:/Users/robaeten/OneDrive - UGent/Gait Classification and References/Data/Subject20/Session1_Proc_Ref/Subj20 - Rec1 - Circular walking (diam 2m)(CCW) - Rep1_1 - ModelOutputs.csv


In [6]:
fileToRead = filesToRead[0]
df_modOut = pd.read_csv(fileToRead)
df_modOut.head(5)

,CentreOfMassX,CentreOfMassY,CentreOfMassZ,CentreOfMassFloorX,CentreOfMassFloorY,CentreOfMassFloorZ,HED_RX,HED_RY,HED_RZ,HED_TX,...,RWristPowerZ,TRX_RX,TRX_RY,TRX_RZ,TRX_TX,TRX_TY,TRX_TZ,TRX_SX,TRX_SY,TRX_SZ
0,-685.074,1459.55,962.095,-685.074,1459.55,0,14.2031,4.63975,304.079,-752.761,...,0.000787,-137.104,98.8116,12.4144,-656.750,1416.39,1343.91,156.484,156.484,156.484
1,-682.739,1448.66,963.851,-682.739,1448.66,0,13.2019,3.97153,304.659,-751.366,...,0.000908,-137.806,97.7899,12.5690,-654.102,1406.35,1345.56,156.499,156.499,156.499
2,-680.301,1437.91,965.572,-680.301,1437.91,0,12.3212,3.25029,305.243,-749.843,...,0.000977,-138.516,96.7624,12.7331,-651.337,1396.48,1347.17,156.518,156.518,156.518
3,-677.762,1427.30,967.241,-677.762,1427.30,0,11.5658,2.48810,305.829,-748.192,...,0.000995,-139.230,95.7345,12.9049,-648.459,1386.76,1348.73,156.541,156.541,156.540
4,-675.122,1416.83,968.841,-675.122,1416.83,0,10.9381,1.69637,306.418,-746.416,...,0.000971,-139.943,94.7111,13.0827,-645.473,1377.18,1350.24,156.565,156.565,156.565


In [7]:
### Remove standing offset from joint angles, prep
to_offset = ["LAnkleAnglesX", "LKneeAnglesX", "LHipAnglesX", "RAnkleAnglesX", "RKneeAnglesX", "RHipAnglesX"]
offset = ["LAnkleAnglesX_offset", "LKneeAnglesX_offset", "LHipAnglesX_offset", 
          "RAnkleAnglesX_offset", "RKneeAnglesX_offset", "RHipAnglesX_offset"]

for file in filesToRead:
    if "- Standing -" in file:
        df_standing = pd.read_csv(file)
        offset_angles = np.nanmean(df_standing[to_offset].to_numpy(), axis=0)
        print(offset_angles)

# # Or if there is to many nans in the standing trial, select some idle time from a Trn file
# file = "C:/Users/robaeten/OneDrive - UGent/Gait Classification and References/Data/Subject6/Session1_Proc_Trn/Subj6 - Rec1 - Normal level walking - Rep1 - ModOut.csv"
# df_standing = pd.read_csv(file)
# arr_standing = df_standing[to_offset].to_numpy()
# offset_angles = np.nanmean(arr_standing[700:, :], axis=0)
# print(offset_angles)

### Filter joint moment and power trajectories, prep
to_filter = ["LAnkleMomentX", "LAnkleMomentY", "LAnkleMomentZ", "LKneeMomentX", "LKneeMomentY", "LKneeMomentZ", 
             "LHipMomentX", "LHipMomentY", "LHipMomentZ", "LAnklePowerX", "LAnklePowerY", "LAnklePowerZ", 
             "LKneePowerX", "LKneePowerY", "LKneePowerZ", "LHipPowerX", "LHipPowerY", "LHipPowerZ", 
             "RAnkleMomentX", "RAnkleMomentY", "RAnkleMomentZ", "RKneeMomentX", "RKneeMomentY", "RKneeMomentZ", 
             "RHipMomentX", "RHipMomentY", "RHipMomentZ", "RAnklePowerX", "RAnklePowerY", "RAnklePowerZ", 
             "RKneePowerX", "RKneePowerY", "RKneePowerZ", "RHipPowerX", "RHipPowerY", "RHipPowerZ"]
filtered = ["LAnkleMomentX_filt", "LAnkleMomentY_filt", "LAnkleMomentZ_filt", "LKneeMomentX_filt", "LKneeMomentY_filt", "LKneeMomentZ_filt", 
            "LHipMomentX_filt", "LHipMomentY_filt", "LHipMomentZ_filt", "LAnklePowerX_filt", "LAnklePowerY_filt", "LAnklePowerZ_filt", 
            "LKneePowerX_filt", "LKneePowerY_filt", "LKneePowerZ_filt", "LHipPowerX_filt", "LHipPowerY_filt", "LHipPowerZ_filt", 
            "RAnkleMomentX_filt", "RAnkleMomentY_filt", "RAnkleMomentZ_filt", "RKneeMomentX_filt", "RKneeMomentY_filt", "RKneeMomentZ_filt", 
            "RHipMomentX_filt", "RHipMomentY_filt", "RHipMomentZ_filt", "RAnklePowerX_filt", "RAnklePowerY_filt", "RAnklePowerZ_filt", 
            "RKneePowerX_filt", "RKneePowerY_filt", "RKneePowerZ_filt", "RHipPowerX_filt", "RHipPowerY_filt", "RHipPowerZ_filt"]

f_sample = 100
f_filter = 6
filter = butter(2, f_filter, btype='low', analog=False, output='sos', fs=f_sample)

[12.96092324  6.97305875  9.10122146 12.78814555  6.48991517  8.79775274]


In [8]:
### Actually do it
for fileToRead in filesToRead:
    df_modOut = pd.read_csv(fileToRead)
    
    for idx, header in enumerate(to_offset):
        arr = df_modOut[header].to_numpy()
        arr_offset = arr-offset_angles[idx]
        df_modOut[offset[idx]] = arr_offset

    for idx, header in enumerate(to_filter):
        arr = df_modOut[header].to_numpy()
        arr_filt = sosfiltfilt(filter, arr, axis=0)
        df_modOut[filtered[idx]] = arr_filt

    df_modOut.to_csv(fileToRead, index=False, float_format="%.6g")